# Training

In [1]:
import sys, os
import pandas as pd
import numpy as np
import setting
import pathSetting
import torch
sys.path.append("../../..")

from Clust.clust.transformation.type.DFToNPArray import transDFtoNP, trans_df_to_np, trans_df_to_np_inf
from Clust.clust.ML.common.common import p1_integratedDataSaving as p1
from Clust.clust.ML.tool import data as ml_data
from Clust.clust.ML.tool import meta as ml_meta
from Clust.clust.ML.tool import model as ml_model

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

cpu is available.


In [2]:
modelConfig={
    "LSTM_cf":{# Case 1. LSTM model (w/o data representation)
        'num_layers': 2,  # recurrent layers의 수, int(default: 2, 범위: 1 이상)
        'hidden_size': 64,  # hidden state의 차원, int(default: 64, 범위: 1 이상)
        'dropout': 0.1,  # dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
        'bidirectional': True,  # 모델의 양방향성 여부, bool(default: True)   
        "lr":0.0001
    },
    "GRU_cf":{# Case 2. GRU model (w/o data representation)
        'num_layers': 2,  # recurrent layers의 수, int(default: 2, 범위: 1 이상)
        'hidden_size': 64,  # hidden state의 차원, int(default: 64, 범위: 1 이상)
        'dropout': 0.1,  # dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
        'bidirectional': True,  # 모델의 양방향성 여부, bool(default: True)
        "lr":0.0001
        
    },
    "CNN_1D_cf":{# Case 3. CNN_1D model (w/o data representation)
        'output_channels': 64, # convolution layer의 output channel, int(default: 64, 범위: 1 이상, 2의 지수로 설정 권장)
        'kernel_size': 3, # convolutional layer의 filter 크기, int(default: 3, 범위: 3 이상, 홀수로 설정 권장)
        'stride': 1, # convolution layer의 stride 크기, int(default: 1, 범위: 1 이상)
        'padding': 0, # padding 크기, int(default: 0, 범위: 0 이상)
        'drop_out': 0.1, # dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
        "lr":0.0001
    },
    "LSTM_FCNs_cf":{#Case 4. LSTM_FCNs model (w/o data representation)
        'num_layers': 1,  # recurrent layers의 수, int(default: 1, 범위: 1 이상)
        'lstm_drop_out': 0.4, # LSTM dropout 확률, float(default: 0.4, 범위: 0 이상 1 이하)
        'fc_drop_out': 0.1, # FC dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
        "lr":0.0001
    },
    "FC_cf":{# Case 5. fully-connected layers (w/ data representation)
        'drop_out': 0.1, # dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
        'bias': True,# bias 사용 여부, bool(default: True)
        "lr":0.0001}  
}

## Data selection

In [3]:
#dataRoot = 'ml_data_integration'
# 1 (p2부분 1. Data Selection)
DataMeta = p1.read_json_data(pathSetting.DataMetaPath)
dataList =  list(DataMeta.keys())

In [4]:
dataList

['trainX_ActionPatternData',
 'trainy_ActionPatternData',
 'testX_ActionPatternData',
 'testy_ActionPatternData']

In [5]:
#2
# dataX
dataName_X = dataList[0]
dataSaveMode_X = DataMeta[dataName_X]["integrationInfo"]["DataSaveMode"]

# datay
dataName_y = dataList[1]
dataSaveMode_y = DataMeta[dataName_y]["integrationInfo"]["DataSaveMode"]

#3
### CSV 로 Local 에 데이터 저장되어 있을 경우
dataFolderName = "data_integrated_result"
current = os.getcwd()
dataFolderPath = os.path.join(current, dataFolderName)

dataX = ml_data.get_saved_integrated_data(dataSaveMode_X, dataName_X, dataFolderPath)
datay = ml_data.get_saved_integrated_data(dataSaveMode_y, dataName_y, dataFolderPath)
integration_freq_sec = DataMeta[dataName_X]["integrationInfo"]["integration_freq_sec"]

## Training Data Preparation

In [6]:
# 2 Training Data Preparation
# 2-1
featureListX= list(dataX.columns)
featureListy= list(datay.columns)
target_col = 'value'

# 2-2
cleanTrainDataParam = 'NoClean'#  Classification, Regression과 같이 X, y가 분리된 경우에는 현재 고정해서 사용해야함

# 2-2-1 cleanTrainDataParam == Clean 일 경우
NaNProcessingParam ={
    "feature_cycle":'Day',
    "feature_cycle_times":1,
    "NanInfoForCleanData":{'type':'num', 'ConsecutiveNanLimit':3, 'totalNaNLimit':30000}
}
# 2-3
scalerParam="scale"
scaleMethod ='minmax'
# 2-4
splitRatio = 0.8
mode = 'Classification'

# 2-5
scalerRootPath_X = os.path.join(pathSetting.scalerRootDir, dataName_X, cleanTrainDataParam)
scalerRootPath_y = os.path.join(pathSetting.scalerRootDir, dataName_X, cleanTrainDataParam)
train_x, val_x, X_scalerFilePath = ml_data.get_train_val_data(dataX, featureListX, scalerRootPath_X, splitRatio, scalerParam, scaleMethod, mode)
train_y, val_y, y_scalerFilePath = ml_data.get_train_val_data(datay, featureListy, scalerRootPath_y, splitRatio, "NoScale", scaleMethod, mode)

['col_0', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7', 'col_8']
Make New scaler File


## Training 

In [7]:
model_list = ["LSTM_cf","GRU_cf", "CNN_1D_cf","LSTM_FCNs_cf"]
model_method = model_list[0]

n_epochs = 2 # 학습 epoch 횟수, int(default: 1000, 범위: 1 이상)
batch_size = 16  # batch 크기, int(default: 16, 범위: 1 이상, 컴퓨터 사양에 적합하게 설정)
num_classes = 6 # class 개수
#targetNames = ['class 0', 'class 1', 'class 2', 'class 3', 'class 4', 'class 5'] ## result_metrics_df 에서 target 이름 설정.... Inference 에서 힘들음

In [8]:
trainParameter = modelConfig[model_method]
trainParameter['device']  = device
trainParameter['n_epochs'] = n_epochs
trainParameter['num_classes'] = num_classes
trainParameter['batch_size'] = batch_size

In [9]:
window_num=0

dim = 3
if model_method == "FC_cf":
    dim = 2
if type(train_x) !=  np.ndarray:
    train_x, train_y = transDFtoNP(train_x, train_y, window_num, dim)
    val_x, val_y = transDFtoNP(val_x, val_y, window_num, dim)

trainParameter['input_size'] = train_x.shape[1]
if dim != 2:
    trainParameter['seq_len']  = train_x.shape[2] 

/home/leezy/anaconda3/lib/python3.8/site-packages/pandas/core/indexes/base.py:5277: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.
  start_slice, end_slice = self.slice_locs(start, end, step=step, kind=kind)


In [10]:
modelTags =["action", "sensor", "classification", "pattern"]
trainDataType = "timeseries"
modelPurpose = "classification"

# 2
trainDataInfo = DataMeta[dataName_X]['integrationInfo']

# model_name = 'New_ClassificationNocleanLSTM_cf'
model_name = 'test_meta_classification'

# 3. 모델을 저장할 파일 패스를 생성한다.
from Clust.clust.transformation.general.dataScaler import encode_hash_style
trainParameter_encode =  encode_hash_style(str(trainParameter))
# trainDataPathList = ["ActionClassificationLSTMCF", dataName_X, trainParameter_encode]
trainDataPathList = [model_name, dataName_X, trainParameter_encode]
# trainDataPathList = [model_name]
modelFilePath = ml_model.get_model_file_path(trainDataPathList, model_method)

./Models/LSTM_cf/test_meta_classification/trainX_ActionPatternData/2c4ed4d8c401715309eb2ae4bcf85d8a/model.pkl


In [11]:
# 4. Training
from Clust.clust.ML.classification.train import ClassificationTrain as CTrain
ctrain = CTrain()
ctrain.set_param(trainParameter)
ctrain.set_model(model_method)
ctrain.set_data(train_x, train_y, val_x, val_y)
ctrain.train()
ctrain.save_best_model(modelFilePath)

Start training model

Epoch 1/2
train Loss: 1.6251 Acc: 0.2821
val Loss: 1.3995 Acc: 0.3630

Training complete in 1m 17s
Best val Acc: 0.460911


## Model Meta Save - mongoDB

In [12]:
from Clust.clust.ingestion.mongo import mongo_client
from Clust.setting import influx_setting_KETI as ins

mongo_client = mongo_client.MongoClient(ins.CLUSTMetaInfo2)

In [13]:
train_meta = modelConfig[model_method]
train_meta['device']  = device
train_meta['n_epochs'] = n_epochs
train_meta['num_classes'] = num_classes
train_meta['batch_size'] = batch_size

In [14]:
model_meta = {
    "trainDataInfo": trainDataInfo,
    "featureList": featureListX,
    "target": featureListy,
    "trainDataType": trainDataType,
    "modelPurpose": modelPurpose,
    "model_method": model_method,
    "modelTags": modelTags,
    "trainDataName": [dataName_X,dataName_y],
    "cleanTrainDataParam":cleanTrainDataParam,
    "NaNProcessingParam":NaNProcessingParam,
    "scalerParam": scalerParam,
    "files":{
            "modelFile":{
                    "fileName":"model.pth",
                    "filePath":modelFilePath
                },
            "XScalerFile":{
                    "fileName":"scaler.pkl",
                    "filePath":X_scalerFilePath       
                },
            "yScalerFile":{
                    "fileName":"scaler.pkl",
                    "filePath":y_scalerFilePath       
                }
        }
}

In [15]:
ml_info_meta = {
    "model_name": model_name,
    "model_meta": model_meta,
    "train_meta": trainParameter
}

In [16]:
ml_meta.save_model_meta_data(mongo_client, ml_info_meta)

========== Data Svae Success ==========
{'model_name': 'test_meta_classification', 'model_meta': {'trainDataInfo': {'startTime': '1992-1-11', 'endTime': '2012-02-27', 'dataInfo': [['bio_action_sensors', 'accelerationTrain']], 'processParam': {'refine_param': {'removeDuplication': {'flag': False}, 'staticFrequency': {'flag': False, 'frequency': None}}, 'outlier_param': {'certainErrorToNaN': {'flag': False}, 'unCertainErrorToNaN': {'flag': False, 'param': {}}}, 'imputation_param': {'flag': False, 'imputation_method': [], 'totalNonNanRatio': 80}}, 'integration_freq_sec': 675, 'cleanParam': 'NoClean', 'DataSaveMode': 'CSV'}, 'featureList': ['col_0', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7', 'col_8'], 'target': ['value'], 'trainDataType': 'timeseries', 'modelPurpose': 'classification', 'model_method': 'LSTM_cf', 'modelTags': ['action', 'sensor', 'classification', 'pattern'], 'trainDataName': ['trainX_ActionPatternData', 'trainy_ActionPatternData'], 'cleanTrainDataParam'

In [18]:
ml_info_meta["model_meta"]

{'trainDataInfo': {'startTime': '1992-1-11',
  'endTime': '2012-02-27',
  'dataInfo': [['bio_action_sensors', 'accelerationTrain']],
  'processParam': {'refine_param': {'removeDuplication': {'flag': False},
    'staticFrequency': {'flag': False, 'frequency': None}},
   'outlier_param': {'certainErrorToNaN': {'flag': False},
    'unCertainErrorToNaN': {'flag': False, 'param': {}}},
   'imputation_param': {'flag': False,
    'imputation_method': [],
    'totalNonNanRatio': 80}},
  'integration_freq_sec': 675,
  'cleanParam': 'NoClean',
  'DataSaveMode': 'CSV'},
 'featureList': ['col_0',
  'col_1',
  'col_2',
  'col_3',
  'col_4',
  'col_5',
  'col_6',
  'col_7',
  'col_8'],
 'target': ['value'],
 'trainDataType': 'timeseries',
 'modelPurpose': 'classification',
 'model_method': 'LSTM_cf',
 'modelTags': ['action', 'sensor', 'classification', 'pattern'],
 'trainDataName': ['trainX_ActionPatternData', 'trainy_ActionPatternData'],
 'cleanTrainDataParam': 'NoClean',
 'NaNProcessingParam': {'f

In [19]:
ml_info_meta["train_meta"]

{'num_layers': 2,
 'hidden_size': 64,
 'dropout': 0.1,
 'bidirectional': True,
 'lr': 0.0001,
 'device': 'cpu',
 'n_epochs': 2,
 'num_classes': 6,
 'batch_size': 16,
 'input_size': 9,
 'seq_len': 128,
 'rnn_type': 'lstm'}